In [1]:
from __future__ import absolute_import, division, print_function
import os, sys, json
current_path = os.path.abspath('.')
parent_path = os.path.dirname(current_path)
sys.path.append(parent_path)
import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torchvision import datasets, transforms
from tqdm import tqdm
from torchinfo import summary

from src.data_loader import *
from src.utils import *

from src.models.simple_cnn import *
from src.models.resnet_models import *
from src.models.dscnn import *
from train_joint import *

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

############################################
########## Plot Style Declaration ##########
# Set the style globally
# Alternatives include bmh, fivethirtyeight, ggplot,
# dark_background, seaborn-deep, etc
# plt.style.use('ggplot')
plt.style.use('seaborn-white')

plt.rcParams['font.family'] = 'times new roman'
# plt.rcParams['font.serif'] = 'Ubuntu'
# plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 15
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 15
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['figure.titlesize'] = 15
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

##################################
########## End of Setup ##########

import tensorflow as tf
from tensorflow import keras
#import tensorflow_addons as tfa
# from tensorflow.keras import layers

In [2]:
class ArgClass():
    def __init__(self):
        super(ArgClass, self).__init__()

args = ArgClass()
args.dataset="cifar10"
args.model_arch = "cnn"
args.batch_size=128
args.test_batch_size=1000
args.percent=[0.8, 0.92, 0.991, 0.93]
args.alpha=5e-4
args.rho=1e-2
args.l1=False
args.l2=False
args.num_pre_epochs=3
args.num_epochs=10
args.num_re_epochs=3
args.lr=1e-3
args.adam_epsilon=1e-8
args.no_cuda=False
args.seed=1
args.save_model=False
args.shuffle=True

args.optimizer_name = 'adam'
args.lr_mode = 'multistep'
args.lr_decay = 0.1
args.lr_decay_epoch = '20,40'
args.target_lr = 1e-8
args.warmup_epochs = 0
args.warmup_lr = 1e-8
args.warmup_mode = 'linear'
args.momentum = 0.9
args.wd = 0.0001
args.gamma_wd_mult = 1.0
args.beta_wd_mult = 1.0
args.bias_wd_mult = 1.0
args.grad_clip = None
args.label_smoothing = False


args.test_fold_l = '[10]'
args.use_one_task = 'false'
args.exp_setup = ''
args.subject_idx = None
args.pretrained=False
args.best_acc = 200.0
args.mixup = False
args.mixup_alpha = 1.0
args.mixup_epoch_tail = 10
args.session = 1
args.test_vote = None

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)
random.seed(args.seed)
np.random.seed(args.seed)
tf.random.set_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# Setting seed for reproducibiltiy


In [3]:
device

device(type='cuda')

In [4]:
print(tf.__version__)
print(keras.__version__)

2.3.1
2.4.0


In [3]:
# MicroNet-AD-K
init_block_channel = 168
init_block_stride = (1,1)
channels = [[192],[192],[192],[192],[192]]
# channels = [[276],[276],[276],[276],[276]]
strides = [2, 1, 1, 2, 2]

def get_pool_size(strides, config):
    pool_size = None
    for stride in strides:
        if stride > 1:
            if pool_size is None:
                pool_size=[np.ceil(config["seq"]/2),np.ceil(config["features"]/2)]
            else:
                pool_size = [np.ceil(pool_size[0]/2),np.ceil(pool_size[1]/2)]
    # return pool_size
    return [np.ceil(pool_size[0]/2),np.ceil(pool_size[1]/2)]


In [4]:
print(tf.keras.backend.image_data_format())
# force channels-first ordering
# tf.keras.backend.set_image_data_format('channels_first')
# print(tf.keras.backend.image_data_format())
# bn_axis=1
# channel_ordering='channels_first'
# force channels-last ordering
tf.keras.backend.set_image_data_format('channels_last')
print(tf.keras.backend.image_data_format())
bn_axis=-1
channel_ordering='channels_last'

channels_last
channels_last


In [28]:
# from keras.datasets import cifar10
# load train and test dataset
def load_dataset():
    # load data
    (trainX, trainY), (testX, testY) = keras.datasets.cifar10.load_data()
    return trainX, trainY, testX, testY

def scale(train, test):
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')

    # normalize
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    
    mean_rgb = (0.4914, 0.4822, 0.4465)
    std_rgb = (0.24703233,0.24348505,0.26158768)
    for i in range(3):
        train_norm[:,:,:,i] = (train_norm[:,:,:,i]-mean_rgb[i])/std_rgb[i]
        test_norm[:,:,:,i] = (test_norm[:,:,:,i]-mean_rgb[i])/std_rgb[i]
    
    return train_norm, test_norm

train_X, train_y, test_X, test_y = load_dataset()

train_X, test_X = scale(train_X, test_X)


In [5]:
# model
import tensorflow_addons as tfa
opt = tfa.optimizers.AdamW(learning_rate=0.001, weight_decay=0.0001)
initializer = tf.keras.initializers.HeNormal()
args.config = DATASET_CONFIGS[args.dataset]
print(args.config)
init_block_kernel=(3,3)


tf.keras.backend.clear_session()
model3 = tf.keras.models.Sequential([
# init_block
  tf.keras.layers.Input(shape=[args.config["seq"],args.config["features"],args.config["in_channels"]]),
  tf.keras.layers.experimental.preprocessing.Resizing(32, 32),
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
  tf.keras.layers.experimental.preprocessing.RandomRotation(factor=0.02),
  tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor=0.2, width_factor=0.2),
  tf.keras.layers.Conv2D(filters=init_block_channel,kernel_size=init_block_kernel,
                        strides=init_block_stride,padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=0.1,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 1st layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[0],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=0.1,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[0][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=0.1,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 2nd layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[1],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=0.1,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[1][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=0.1,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 3rd layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[2],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=0.1,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[2][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=0.1,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 4th layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[3],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=0.1,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[3][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=0.1,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 5th layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[4],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=0.1,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[4][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=0.1,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# Note: Add dropout layer after all blocks and before pooling
  tf.keras.layers.Dropout(rate=0.4),
  tf.keras.layers.AveragePooling2D(pool_size=get_pool_size(strides, args.config),data_format=channel_ordering),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(units=args.config["classes"],
                       kernel_initializer=initializer)
])

model3.compile(
    optimizer=opt,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

{'in_channels': 3, 'features': 32, 'seq': 32, 'classes': 10}


In [6]:
model3.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resizing (Resizing)          (None, 32, 32, 3)         0         
_________________________________________________________________
random_flip (RandomFlip)     (None, 32, 32, 3)         0         
_________________________________________________________________
random_rotation (RandomRotat (None, 32, 32, 3)         0         
_________________________________________________________________
random_zoom (RandomZoom)     (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 168)       4536      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 168)       672       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 168)       0

In [20]:
history = model3.fit(
      x=train_X,
      y=train_y,
      batch_size=128,
      epochs=80,
      validation_data=(test_X, test_y)
)

Epoch 1/80
391/391 [==============================] - 11s 28ms/step - loss: 1.6016 - sparse_categorical_accuracy: 0.4132 - val_loss: 1.3994 - val_sparse_categorical_accuracy: 0.5097
Epoch 2/80
391/391 [==============================] - 11s 27ms/step - loss: 1.2106 - sparse_categorical_accuracy: 0.5649 - val_loss: 1.0899 - val_sparse_categorical_accuracy: 0.6134
Epoch 3/80
391/391 [==============================] - 11s 27ms/step - loss: 1.0276 - sparse_categorical_accuracy: 0.6337 - val_loss: 1.0377 - val_sparse_categorical_accuracy: 0.6370
Epoch 4/80
391/391 [==============================] - 11s 27ms/step - loss: 0.9125 - sparse_categorical_accuracy: 0.6787 - val_loss: 0.8635 - val_sparse_categorical_accuracy: 0.6956
Epoch 5/80
391/391 [==============================] - 11s 27ms/step - loss: 0.8258 - sparse_categorical_accuracy: 0.7089 - val_loss: 0.8090 - val_sparse_categorical_accuracy: 0.7158
Epoch 6/80
391/391 [==============================] - 10s 27ms/step - loss: 0.7690 - spars

Epoch 46/80
391/391 [==============================] - 11s 27ms/step - loss: 0.3299 - sparse_categorical_accuracy: 0.8846 - val_loss: 0.4954 - val_sparse_categorical_accuracy: 0.8369
Epoch 47/80
391/391 [==============================] - 11s 27ms/step - loss: 0.3317 - sparse_categorical_accuracy: 0.8830 - val_loss: 0.4734 - val_sparse_categorical_accuracy: 0.8430
Epoch 48/80
391/391 [==============================] - 11s 27ms/step - loss: 0.3269 - sparse_categorical_accuracy: 0.8856 - val_loss: 0.4305 - val_sparse_categorical_accuracy: 0.8619
Epoch 49/80
391/391 [==============================] - 11s 27ms/step - loss: 0.3247 - sparse_categorical_accuracy: 0.8876 - val_loss: 0.4437 - val_sparse_categorical_accuracy: 0.8506
Epoch 50/80
391/391 [==============================] - 10s 27ms/step - loss: 0.3238 - sparse_categorical_accuracy: 0.8871 - val_loss: 0.5073 - val_sparse_categorical_accuracy: 0.8361
Epoch 51/80
391/391 [==============================] - 10s 27ms/step - loss: 0.3272 -

In [21]:
model3.compile(
    optimizer=tfa.optimizers.AdamW(learning_rate=0.0001, weight_decay=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
history = model3.fit(
      x=train_X,
      y=train_y,
      batch_size=128,
      epochs=20,
      validation_data=(test_X, test_y)
)

Epoch 1/20
391/391 [==============================] - 11s 28ms/step - loss: 0.2202 - sparse_categorical_accuracy: 0.9255 - val_loss: 0.3896 - val_sparse_categorical_accuracy: 0.8722
Epoch 2/20
391/391 [==============================] - 10s 27ms/step - loss: 0.1906 - sparse_categorical_accuracy: 0.9383 - val_loss: 0.3779 - val_sparse_categorical_accuracy: 0.8751
Epoch 3/20
391/391 [==============================] - 11s 27ms/step - loss: 0.1807 - sparse_categorical_accuracy: 0.9425 - val_loss: 0.3760 - val_sparse_categorical_accuracy: 0.8752
Epoch 4/20
391/391 [==============================] - 10s 27ms/step - loss: 0.1755 - sparse_categorical_accuracy: 0.9457 - val_loss: 0.3409 - val_sparse_categorical_accuracy: 0.8864
Epoch 5/20
391/391 [==============================] - 11s 27ms/step - loss: 0.1720 - sparse_categorical_accuracy: 0.9474 - val_loss: 0.3426 - val_sparse_categorical_accuracy: 0.8847
Epoch 6/20
391/391 [==============================] - 11s 27ms/step - loss: 0.1676 - spars

In [16]:
# model
import tensorflow_addons as tfa
opt = tfa.optimizers.AdamW(learning_rate=0.001, weight_decay=0.0001)
initializer = tf.keras.initializers.HeNormal()
args.config = DATASET_CONFIGS[args.dataset]
print(args.config)
init_block_kernel=(3,3)
mmt=0.1

tf.keras.backend.clear_session()
model3 = tf.keras.models.Sequential([
# init_block
  tf.keras.layers.Input(shape=[args.config["seq"],args.config["features"],args.config["in_channels"]]),
  tf.keras.layers.experimental.preprocessing.Resizing(32, 32),
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
  tf.keras.layers.experimental.preprocessing.RandomRotation(factor=0.02),
  tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor=0.2, width_factor=0.2),
  tf.keras.layers.Conv2D(filters=init_block_channel,kernel_size=init_block_kernel,
                        strides=init_block_stride,padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 1st layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[0],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[0][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 2nd layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[1],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[1][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 3rd layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[2],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[2][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 4th layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[3],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[3][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 5th layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[4],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=0.1,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[4][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# Note: Add dropout layer after all blocks and before pooling
  tf.keras.layers.Dropout(rate=0.4),
  tf.keras.layers.AveragePooling2D(pool_size=get_pool_size(strides, args.config),data_format=channel_ordering),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(units=args.config["classes"],
                       kernel_initializer=initializer)
])

model3.compile(
    optimizer=opt,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

{'in_channels': 3, 'features': 32, 'seq': 32, 'classes': 10}


In [ ]:
model3.summary()

In [17]:
history = model3.fit(
      x=train_X,
      y=train_y,
      batch_size=32,
      epochs=80,
      validation_data=(test_X, test_y)
)

Epoch 1/80
1563/1563 [==============================] - 17s 11ms/step - loss: 1.5056 - sparse_categorical_accuracy: 0.4504 - val_loss: 1.1011 - val_sparse_categorical_accuracy: 0.6010
Epoch 2/80
1563/1563 [==============================] - 17s 11ms/step - loss: 1.1024 - sparse_categorical_accuracy: 0.6062 - val_loss: 1.0331 - val_sparse_categorical_accuracy: 0.6401
Epoch 3/80
1563/1563 [==============================] - 17s 11ms/step - loss: 0.9415 - sparse_categorical_accuracy: 0.6687 - val_loss: 1.0602 - val_sparse_categorical_accuracy: 0.6418
Epoch 4/80
1563/1563 [==============================] - 17s 11ms/step - loss: 0.8490 - sparse_categorical_accuracy: 0.7007 - val_loss: 0.8311 - val_sparse_categorical_accuracy: 0.7093
Epoch 5/80
1563/1563 [==============================] - 17s 11ms/step - loss: 0.7797 - sparse_categorical_accuracy: 0.7257 - val_loss: 0.7945 - val_sparse_categorical_accuracy: 0.7250
Epoch 6/80
1563/1563 [==============================] - 17s 11ms/step - loss: 0.

1563/1563 [==============================] - 15s 10ms/step - loss: 0.5508 - sparse_categorical_accuracy: 0.8106 - val_loss: 0.9278 - val_sparse_categorical_accuracy: 0.7187
Epoch 46/80
1563/1563 [==============================] - 15s 10ms/step - loss: 0.5502 - sparse_categorical_accuracy: 0.8093 - val_loss: 0.6423 - val_sparse_categorical_accuracy: 0.7838
Epoch 47/80
1563/1563 [==============================] - 15s 10ms/step - loss: 0.5466 - sparse_categorical_accuracy: 0.8122 - val_loss: 0.6523 - val_sparse_categorical_accuracy: 0.7838
Epoch 48/80
1563/1563 [==============================] - 15s 10ms/step - loss: 0.5503 - sparse_categorical_accuracy: 0.8107 - val_loss: 0.6173 - val_sparse_categorical_accuracy: 0.7901
Epoch 49/80
1563/1563 [==============================] - 16s 10ms/step - loss: 0.5503 - sparse_categorical_accuracy: 0.8108 - val_loss: 0.6384 - val_sparse_categorical_accuracy: 0.7830
Epoch 50/80
1563/1563 [==============================] - 16s 10ms/step - loss: 0.5533 -

In [18]:
model3.compile(
    optimizer=tfa.optimizers.AdamW(learning_rate=0.0005, weight_decay=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
history = model3.fit(
      x=train_X,
      y=train_y,
      batch_size=32,
      epochs=1,
      validation_data=(test_X, test_y)
)

1563/1563 [==============================] - 18s 12ms/step - loss: 0.4600 - sparse_categorical_accuracy: 0.8431 - val_loss: 0.4872 - val_sparse_categorical_accuracy: 0.8414


In [19]:
model3.compile(
    optimizer=tfa.optimizers.AdamW(learning_rate=0.0001, weight_decay=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
history = model3.fit(
      x=train_X,
      y=train_y,
      batch_size=32,
      epochs=2,
      validation_data=(test_X, test_y)
)

Epoch 1/2
1563/1563 [==============================] - 18s 11ms/step - loss: 0.3658 - sparse_categorical_accuracy: 0.8807 - val_loss: 0.4433 - val_sparse_categorical_accuracy: 0.8494
Epoch 2/2
1563/1563 [==============================] - 17s 11ms/step - loss: 0.3805 - sparse_categorical_accuracy: 0.8794 - val_loss: 0.4658 - val_sparse_categorical_accuracy: 0.8455


In [8]:
# model
import tensorflow_addons as tfa
opt = tfa.optimizers.AdamW(learning_rate=0.001, weight_decay=0.0001)
initializer = tf.keras.initializers.HeNormal()
args.config = DATASET_CONFIGS[args.dataset]
print(args.config)
init_block_kernel=(3,3)
mmt=0.2

tf.keras.backend.clear_session()
model3 = tf.keras.models.Sequential([
# init_block
  tf.keras.layers.Input(shape=[args.config["seq"],args.config["features"],args.config["in_channels"]]),
  tf.keras.layers.experimental.preprocessing.Resizing(32, 32),
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
  tf.keras.layers.experimental.preprocessing.RandomRotation(factor=0.02),
  tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor=0.2, width_factor=0.2),
  tf.keras.layers.Conv2D(filters=init_block_channel,kernel_size=init_block_kernel,
                        strides=init_block_stride,padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 1st layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[0],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[0][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 2nd layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[1],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[1][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 3rd layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[2],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[2][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 4th layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[3],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[3][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 5th layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[4],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=0.1,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[4][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# Note: Add dropout layer after all blocks and before pooling
  tf.keras.layers.Dropout(rate=0.4),
  tf.keras.layers.AveragePooling2D(pool_size=get_pool_size(strides, args.config),data_format=channel_ordering),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(units=args.config["classes"],
                       kernel_initializer=initializer)
])

model3.compile(
    optimizer=opt,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

{'in_channels': 3, 'features': 32, 'seq': 32, 'classes': 10}


In [9]:
history = model3.fit(
      x=train_X,
      y=train_y,
      batch_size=32,
      epochs=80,
      validation_data=(test_X, test_y)
)

Epoch 1/40
1563/1563 [==============================] - 17s 11ms/step - loss: 1.4994 - sparse_categorical_accuracy: 0.4533 - val_loss: 1.1115 - val_sparse_categorical_accuracy: 0.5963
Epoch 2/40
1563/1563 [==============================] - 17s 11ms/step - loss: 1.0945 - sparse_categorical_accuracy: 0.6084 - val_loss: 1.0083 - val_sparse_categorical_accuracy: 0.6527
Epoch 3/40
1563/1563 [==============================] - 17s 11ms/step - loss: 0.9328 - sparse_categorical_accuracy: 0.6712 - val_loss: 1.0028 - val_sparse_categorical_accuracy: 0.6516
Epoch 4/40
1563/1563 [==============================] - 17s 11ms/step - loss: 0.8436 - sparse_categorical_accuracy: 0.7035 - val_loss: 0.8492 - val_sparse_categorical_accuracy: 0.7128
Epoch 5/40
1563/1563 [==============================] - 17s 11ms/step - loss: 0.7817 - sparse_categorical_accuracy: 0.7283 - val_loss: 0.7566 - val_sparse_categorical_accuracy: 0.7405
Epoch 6/40
1563/1563 [==============================] - 17s 11ms/step - loss: 0.

In [10]:
model3.compile(
    optimizer=tfa.optimizers.AdamW(learning_rate=0.0005, weight_decay=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
history = model3.fit(
      x=train_X,
      y=train_y,
      batch_size=32,
      epochs=1,
      validation_data=(test_X, test_y)
)

1563/1563 [==============================] - 17s 11ms/step - loss: 0.4562 - sparse_categorical_accuracy: 0.8431 - val_loss: 0.4600 - val_sparse_categorical_accuracy: 0.8416


In [11]:
model3.compile(
    optimizer=tfa.optimizers.AdamW(learning_rate=0.0001, weight_decay=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
history = model3.fit(
      x=train_X,
      y=train_y,
      batch_size=32,
      epochs=2,
      validation_data=(test_X, test_y)
)

Epoch 1/2
1563/1563 [==============================] - 17s 11ms/step - loss: 0.3570 - sparse_categorical_accuracy: 0.8849 - val_loss: 0.4239 - val_sparse_categorical_accuracy: 0.8545
Epoch 2/2
1563/1563 [==============================] - 17s 11ms/step - loss: 0.3748 - sparse_categorical_accuracy: 0.8815 - val_loss: 0.4363 - val_sparse_categorical_accuracy: 0.8525


In [12]:
# model
import tensorflow_addons as tfa
opt = tfa.optimizers.AdamW(learning_rate=0.001, weight_decay=0.0001)
initializer = tf.keras.initializers.HeNormal()
args.config = DATASET_CONFIGS[args.dataset]
print(args.config)
init_block_kernel=(3,3)
mmt=0.2

tf.keras.backend.clear_session()
model4 = tf.keras.models.Sequential([
# init_block
  tf.keras.layers.Input(shape=[args.config["seq"],args.config["features"],args.config["in_channels"]]),
  tf.keras.layers.experimental.preprocessing.Resizing(32, 32),
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
  tf.keras.layers.experimental.preprocessing.RandomRotation(factor=0.02),
  tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor=0.2, width_factor=0.2),
  tf.keras.layers.Conv2D(filters=init_block_channel,kernel_size=init_block_kernel,
                        strides=init_block_stride,padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 1st layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[0],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[0][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 2nd layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[1],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[1][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 3rd layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[2],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[2][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 4th layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[3],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[3][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 5th layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[4],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=0.1,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[4][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# Note: Add dropout layer after all blocks and before pooling
  tf.keras.layers.Dropout(rate=0.4),
  tf.keras.layers.AveragePooling2D(pool_size=get_pool_size(strides, args.config),data_format=channel_ordering),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(units=args.config["classes"],
                       kernel_initializer=initializer)
])

model4.compile(
    optimizer=opt,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

{'in_channels': 3, 'features': 32, 'seq': 32, 'classes': 10}


In [13]:
history = model4.fit(
      x=train_X,
      y=train_y,
      batch_size=32,
      epochs=80,
      validation_data=(test_X, test_y)
)

Epoch 1/80
1563/1563 [==============================] - 18s 11ms/step - loss: 1.5228 - sparse_categorical_accuracy: 0.4446 - val_loss: 1.1274 - val_sparse_categorical_accuracy: 0.5926
Epoch 2/80
1563/1563 [==============================] - 17s 11ms/step - loss: 1.1228 - sparse_categorical_accuracy: 0.5978 - val_loss: 1.0315 - val_sparse_categorical_accuracy: 0.6389
Epoch 3/80
1563/1563 [==============================] - 17s 11ms/step - loss: 0.9646 - sparse_categorical_accuracy: 0.6554 - val_loss: 1.0055 - val_sparse_categorical_accuracy: 0.6554
Epoch 4/80
1563/1563 [==============================] - 17s 11ms/step - loss: 0.8664 - sparse_categorical_accuracy: 0.6950 - val_loss: 0.8279 - val_sparse_categorical_accuracy: 0.7123
Epoch 5/80
1563/1563 [==============================] - 17s 11ms/step - loss: 0.8033 - sparse_categorical_accuracy: 0.7200 - val_loss: 0.8014 - val_sparse_categorical_accuracy: 0.7293
Epoch 6/80
1563/1563 [==============================] - 17s 11ms/step - loss: 0.

1563/1563 [==============================] - 17s 11ms/step - loss: 0.5613 - sparse_categorical_accuracy: 0.8059 - val_loss: 0.6687 - val_sparse_categorical_accuracy: 0.7715
Epoch 46/80
1563/1563 [==============================] - 17s 11ms/step - loss: 0.5548 - sparse_categorical_accuracy: 0.8095 - val_loss: 0.5771 - val_sparse_categorical_accuracy: 0.8075
Epoch 47/80
1563/1563 [==============================] - 17s 11ms/step - loss: 0.5547 - sparse_categorical_accuracy: 0.8094 - val_loss: 0.5997 - val_sparse_categorical_accuracy: 0.7964
Epoch 48/80
1563/1563 [==============================] - 17s 11ms/step - loss: 0.5576 - sparse_categorical_accuracy: 0.8077 - val_loss: 0.5670 - val_sparse_categorical_accuracy: 0.8042
Epoch 49/80
1563/1563 [==============================] - 17s 11ms/step - loss: 0.5537 - sparse_categorical_accuracy: 0.8113 - val_loss: 0.5433 - val_sparse_categorical_accuracy: 0.8163
Epoch 50/80
1563/1563 [==============================] - 17s 11ms/step - loss: 0.5525 -

In [14]:
model4.compile(
    optimizer=tfa.optimizers.AdamW(learning_rate=0.0005, weight_decay=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
history = model4.fit(
      x=train_X,
      y=train_y,
      batch_size=32,
      epochs=1,
      validation_data=(test_X, test_y)
)

1563/1563 [==============================] - 17s 11ms/step - loss: 0.4712 - sparse_categorical_accuracy: 0.8377 - val_loss: 0.4998 - val_sparse_categorical_accuracy: 0.8275


In [15]:
model4.compile(
    optimizer=tfa.optimizers.AdamW(learning_rate=0.0001, weight_decay=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
history = model4.fit(
      x=train_X,
      y=train_y,
      batch_size=32,
      epochs=2,
      validation_data=(test_X, test_y)
)

Epoch 1/2
1563/1563 [==============================] - 17s 11ms/step - loss: 0.3801 - sparse_categorical_accuracy: 0.8771 - val_loss: 0.4219 - val_sparse_categorical_accuracy: 0.8552
Epoch 2/2
1563/1563 [==============================] - 17s 11ms/step - loss: 0.3883 - sparse_categorical_accuracy: 0.8774 - val_loss: 0.4737 - val_sparse_categorical_accuracy: 0.8367


# new arch

In [31]:
# MicroNet-AD-K
init_block_channel = 168
init_block_stride = (1,1)
# channels = [[192],[192],[192],[192],[192]]
channels = [[276],[276],[276],[276],[276]]
strides = [2, 1, 1, 2, 2]

def get_pool_size(strides, config):
    pool_size = None
    for stride in strides:
        if stride > 1:
            if pool_size is None:
                pool_size=[np.ceil(config["seq"]/2),np.ceil(config["features"]/2)]
            else:
                pool_size = [np.ceil(pool_size[0]/2),np.ceil(pool_size[1]/2)]
    # return pool_size
    return [np.ceil(pool_size[0]/2),np.ceil(pool_size[1]/2)]


In [33]:
# model
import tensorflow_addons as tfa
opt = tfa.optimizers.AdamW(learning_rate=0.001, weight_decay=0.0001)
initializer = tf.keras.initializers.HeNormal()
args.config = DATASET_CONFIGS[args.dataset]
print(args.config)
init_block_kernel=(3,3)
mmt=0.1

tf.keras.backend.clear_session()
model3 = tf.keras.models.Sequential([
# init_block
  tf.keras.layers.Input(shape=[args.config["seq"],args.config["features"],args.config["in_channels"]]),
  tf.keras.layers.experimental.preprocessing.Resizing(32, 32),
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
  tf.keras.layers.experimental.preprocessing.RandomRotation(factor=0.02),
  tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor=0.2, width_factor=0.2),
  tf.keras.layers.Conv2D(filters=init_block_channel,kernel_size=init_block_kernel,
                        strides=init_block_stride,padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 1st layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[0],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[0][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 2nd layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[1],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[1][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 3rd layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[2],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[2][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 4th layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[3],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[3][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# 5th layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[4],padding='same',use_bias=False,
                                 kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=0.1,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[4][0],kernel_size=(1,1),
                        padding='same',use_bias=False,
                        kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(axis=bn_axis,momentum=mmt,epsilon=0.00001),
  tf.keras.layers.Activation('relu'),
# Note: Add dropout layer after all blocks and before pooling
  tf.keras.layers.Dropout(rate=0.4),
  tf.keras.layers.AveragePooling2D(pool_size=get_pool_size(strides, args.config),data_format=channel_ordering),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(units=args.config["classes"],
                       kernel_initializer=initializer)
])

#
model3.compile(
    optimizer=opt,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
history = model3.fit(
      x=train_X,
      y=train_y,
      batch_size=32,
      epochs=50,
      validation_data=(test_X, test_y)
)
#
model3.compile(
    optimizer=tfa.optimizers.AdamW(learning_rate=0.0005, weight_decay=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
history = model3.fit(
      x=train_X,
      y=train_y,
      batch_size=32,
      epochs=1,
      validation_data=(test_X, test_y)
)
#
model3.compile(
    optimizer=tfa.optimizers.AdamW(learning_rate=0.0001, weight_decay=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
history = model3.fit(
      x=train_X,
      y=train_y,
      batch_size=32,
      epochs=2,
      validation_data=(test_X, test_y)
)

{'in_channels': 3, 'features': 32, 'seq': 32, 'classes': 10}
Epoch 1/50
1563/1563 [==============================] - 20s 13ms/step - loss: 1.4713 - sparse_categorical_accuracy: 0.4665 - val_loss: 1.0642 - val_sparse_categorical_accuracy: 0.6220
Epoch 2/50
1563/1563 [==============================] - 20s 12ms/step - loss: 1.0451 - sparse_categorical_accuracy: 0.6319 - val_loss: 0.9381 - val_sparse_categorical_accuracy: 0.6779
Epoch 3/50
1563/1563 [==============================] - 20s 13ms/step - loss: 0.8882 - sparse_categorical_accuracy: 0.6872 - val_loss: 0.9478 - val_sparse_categorical_accuracy: 0.6703
Epoch 4/50
1563/1563 [==============================] - 20s 13ms/step - loss: 0.7948 - sparse_categorical_accuracy: 0.7211 - val_loss: 0.8299 - val_sparse_categorical_accuracy: 0.7199
Epoch 5/50
1563/1563 [==============================] - 20s 13ms/step - loss: 0.7273 - sparse_categorical_accuracy: 0.7457 - val_loss: 0.7617 - val_sparse_categorical_accuracy: 0.7385
Epoch 6/50
1563/156

1563/1563 [==============================] - 20s 13ms/step - loss: 0.5257 - sparse_categorical_accuracy: 0.8200 - val_loss: 0.6827 - val_sparse_categorical_accuracy: 0.7749
Epoch 46/50
1563/1563 [==============================] - 19s 12ms/step - loss: 0.5233 - sparse_categorical_accuracy: 0.8184 - val_loss: 0.6395 - val_sparse_categorical_accuracy: 0.7900
Epoch 47/50
1563/1563 [==============================] - 20s 13ms/step - loss: 0.5157 - sparse_categorical_accuracy: 0.8241 - val_loss: 0.5763 - val_sparse_categorical_accuracy: 0.8092
Epoch 48/50
1563/1563 [==============================] - 20s 12ms/step - loss: 0.5242 - sparse_categorical_accuracy: 0.8204 - val_loss: 0.5642 - val_sparse_categorical_accuracy: 0.8107
Epoch 49/50
1563/1563 [==============================] - 20s 13ms/step - loss: 0.5191 - sparse_categorical_accuracy: 0.8222 - val_loss: 0.5746 - val_sparse_categorical_accuracy: 0.8061
Epoch 50/50
1563/1563 [==============================] - 20s 13ms/step - loss: 0.4335 -

In [23]:
model3.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resizing (Resizing)          (None, 32, 32, 3)         0         
_________________________________________________________________
random_flip (RandomFlip)     (None, 32, 32, 3)         0         
_________________________________________________________________
random_rotation (RandomRotat (None, 32, 32, 3)         0         
_________________________________________________________________
random_zoom (RandomZoom)     (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 168)       4536      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 168)       672       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 168)       0